# Fase 2: Comprensión de los Datos

## Importación de librerías y carga de datasets

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import kedro

In [5]:
from pathlib import Path
import sys, tomllib
from kedro.framework.project import configure_project
from kedro.framework.session import KedroSession

# Detecta la raíz del proyecto y el package_name
project_path = Path.cwd() if (Path.cwd() / "pyproject.toml").exists() else Path.cwd().parent
with open(project_path / "pyproject.toml", "rb") as f:
    package_name = tomllib.load(f)["tool"]["kedro"]["package_name"]

# Asegura que src/ esté importable
sys.path.insert(0, str(project_path / "src"))

# Inicializa Kedro
configure_project(package_name)
session = KedroSession.create(project_path=project_path)
context = session.load_context()
catalog = context.catalog

# Carga todos los CSV y muestra shapes
names = ["releases","genres", "countries"]
dfs = {name: catalog.load(name) for name in names}
for name in dfs:
    print(f"dataset {name} cargado")

[09/13/25 20:16:13] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=881412;file://C:\Users\edugo\Desktop\Workspace\Universidad\2025-II\MachineLearning\letterboxdml\.venv\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=213447;file://C:\Users\edugo\Desktop\Workspace\Universidad\2025-II\MachineLearning\letterboxdml\.venv\Lib\site-packages\kedro_telemetry\plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

                    INFO     Loading data from releases (CSVDataset)...                        ]8;id=509229;file://C:\Users\edugo\Desktop\Workspace\Universidad\2025-II\MachineLearning\letterboxdml\.venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=359268;file://C:\Users\edugo\Desktop\Workspace\Universidad\2025-II\MachineLearning\letterboxdml\.venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[09/13/25 20:16:14] INFO     Loading data from genres (CSVDataset)...                          ]8;id=991126;file://C:\Users\edugo\Desktop\Workspace\Universidad\2025-II\MachineLearning\letterboxdml\.venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=714818;file://C:\Users\edugo\Desktop\Workspace\Universidad\2025-II\MachineLearning\letterboxdml\.venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

                    INFO     Loading data from countries (CSVDataset)...                       ]8;id=683214;file://C:\Users\edugo\Desktop\Workspace\Universidad\2025-II\MachineLearning\letterboxdml\.venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=451819;file://C:\Users\edugo\Desktop\Workspace\Universidad\2025-II\MachineLearning\letterboxdml\.venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

dataset releases cargado
dataset genres cargado
dataset countries cargado


## Recolectar datos iniciales (3 datasets)

In [8]:
from IPython.display import display

for name in ["releases", "genres", "countries"]:
    df = dfs[name]  # ya cargado en tu celda anterior
    print(f"\n=== DATASET: {name} ===")
    print("shape:", df.shape)
    display(df.head(5))

    # Tipos de datos de todas las variables (columnas)
    schema = pd.DataFrame({
        "column": df.columns,
        "dtype": df.dtypes.astype(str)
    })
    print("dtypes por columna:")
    display(schema)


=== DATASET: releases ===
shape: (1332782, 5)


,id,country,date,type,rating
0,1000001,Andorra,2023-07-21,Theatrical,NaN
1,1000001,Argentina,2023-07-20,Theatrical,ATP
2,1000001,Australia,2023-07-19,Theatrical,PG
3,1000001,Australia,2023-10-01,Digital,PG
4,1000001,Austria,2023-07-20,Theatrical,NaN


dtypes por columna:


,column,dtype
id,id,int64
country,country,object
date,date,object
type,type,object
rating,rating,object



=== DATASET: genres ===
shape: (1046849, 2)


...

,column,dtype
id,id,int64
genre,genre,object



=== DATASET: countries ===
shape: (693476, 2)


,id,country
0,1000001,UK
1,1000001,USA
2,1000002,South Korea
3,1000003,USA
4,1000004,Germany


dtypes por columna:


,column,dtype
id,id,int64
country,country,object


## Descripción de los datos

### Resumen de los dataset

*   **releases** — _(1,332,782 filas; 5 columnas)_: registros de **eventos de estreno** por película y país, con tipo de lanzamiento y clasificación.
    
*   **genres** — _(1,046,849 filas; 2 columnas)_: asignaciones **película–género** (una película puede tener varios géneros).
    
*   **countries** — _(693,476 filas; 2 columnas)_: asignaciones **película–país** (una película puede estar asociada a múltiples países).
    
Granularidad y llaves
    
*   En los tres datasets, **id** es el **identificador de película** (clave para unir).

### Descripción de variables por dataset

**releases (id, country, date, type, rating)**

*   **id** _(int64)_: identificador de película.
    
*   **country** _(object)_: país donde ocurre el **evento de estreno** (ej.: _Andorra, Argentina, Australia, Austria_).
    
*   **date** _(object)_: **fecha del estreno** (actualmente como texto; requiere parseo a datetime para análisis temporal).
    
*   **type** _(object)_: **tipo de lanzamiento** (ej.: _Theatrical, Digital_).
    
*   **rating** _(object / categórico, con ausentes)_: **clasificación**/certificación local (ej.: _ATP, PG_; también puede estar vacío/NaN).
    
**genres (id, genre)**

*   **id** _(int64)_: identificador de película.
    
*   **genre** _(object)_: nombre del **género** (ej.: _Comedy, Adventure, Thriller, Drama_).
    
**countries (id, country)**

*   **id** _(int64)_: identificador de película.
    
*   **country** _(object)_: país **asociado** a la película (ej.: _USA, UK, Germany, South Korea_).

# EDA

In [9]:
from IPython.display import display

releases  = dfs["releases"].copy()
genres    = dfs["genres"].copy()
countries = dfs["countries"].copy()

for name, df in {"releases": releases, "genres": genres, "countries": countries}.items():
    print(f"\n=== DATASET: {name} ===")
    print("shape:", df.shape)
    display(df.head(5))
    schema = pd.DataFrame({"column": df.columns, "dtype": df.dtypes.astype(str)})
    print("dtypes por columna:")
    display(schema)



=== DATASET: releases ===
shape: (1332782, 5)


,id,country,date,type,rating
0,1000001,Andorra,2023-07-21,Theatrical,NaN
1,1000001,Argentina,2023-07-20,Theatrical,ATP
2,1000001,Australia,2023-07-19,Theatrical,PG
3,1000001,Australia,2023-10-01,Digital,PG
4,1000001,Austria,2023-07-20,Theatrical,NaN


dtypes por columna:


,column,dtype
id,id,int64
country,country,object
date,date,object
type,type,object
rating,rating,object



=== DATASET: genres ===
shape: (1046849, 2)


...

,column,dtype
id,id,int64
genre,genre,object



=== DATASET: countries ===
shape: (693476, 2)


,id,country
0,1000001,UK
1,1000001,USA
2,1000002,South Korea
3,1000003,USA
4,1000004,Germany


dtypes por columna:


,column,dtype
id,id,int64
country,country,object


Se crean copias para no modificar los DataFrames originales guardados en dfs y confirmamos que devuelven lo mismo que en el punto "Recolectar datos iniciales" (esto porque según la rúbrica de la evaluación el limpiado de datos va en la fase 3)

## Medidas de tendencia central por dataset

In [25]:
# Medidas de tendencia central (media, mediana, moda) por dataset
import pandas as pd
import numpy as np
from IPython.display import display

def central_tendency_by_dataset(df, dataset_name, date_col=None):
    print("="*40)
    print(f"\n=== Medidas de tendencia central — {dataset_name} ===")
    print("="*40)
    
    # Numéricas: media y mediana (excluye 'id')
    num = df.select_dtypes(include=[np.number]).drop(columns=["id"], errors="ignore")
    if num.shape[1] > 0:
        out_num = pd.DataFrame({
            "mean": num.mean(numeric_only=True),
            "median": num.median(numeric_only=True)
        })
        print("Numéricas (media, mediana):")
        display(out_num.round(4))
    else:
        print("Numéricas: (sin columnas numéricas relevantes)")
    
    # Categóricas: moda (valor más frecuente)
    cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
    if cat_cols:
        rows, n = [], len(df)
        for c in cat_cols:
            vc = df[c].value_counts(dropna=False)
            mode_val = vc.index[0] if len(vc) else None
            mode_freq = int(vc.iloc[0]) if len(vc) else 0
            mode_share = (mode_freq / n) if n else np.nan
            rows.append({"column": c, "mode": mode_val, "mode_freq": mode_freq, "mode_share": round(mode_share, 4)})
        print("Categóricas (moda, frecuencia, participación):")
        display(pd.DataFrame(rows).sort_values("mode_share", ascending=False))
    else:
        print("Categóricas: (sin columnas categóricas)")
    
    # Fechas: mediana (si aplica)
    if date_col and date_col in df.columns:
        s = pd.to_datetime(df[date_col], errors="coerce", utc=True).dt.tz_localize(None)
        med = s.median() if s.notna().any() else None
        print(f"Mediana de fecha ({date_col}): {med}")

# Usa los DataFrames ya cargados en tu sesión:
releases  = dfs["releases"].copy()
genres    = dfs["genres"].copy()
countries = dfs["countries"].copy()

central_tendency_by_dataset(releases,  "releases",  date_col="date")
central_tendency_by_dataset(genres,    "genres")
central_tendency_by_dataset(countries, "countries")



=== Medidas de tendencia central — releases ===
Numéricas: (sin columnas numéricas relevantes)
Categóricas (moda, frecuencia, participación):


,column,mode,mode_freq,mode_share
3,rating,NaN,998802,0.7494
2,type,Theatrical,750043,0.5628
0,country,USA,320901,0.2408
1,date,2006-01-01,3050,0.0023


Mediana de fecha (date): 2012-10-23 00:00:00

=== Medidas de tendencia central — genres ===
Numéricas: (sin columnas numéricas relevantes)
Categóricas (moda, frecuencia, participación):


,column,mode,mode_freq,mode_share
0,genre,Drama,232201,0.2218



=== Medidas de tendencia central — countries ===
Numéricas: (sin columnas numéricas relevantes)
Categóricas (moda, frecuencia, participación):


,column,mode,mode_freq,mode_share
0,country,USA,174489,0.2516


**Releases**

*   **Numéricas:** no hay columnas numéricas relevantes (excluimos id), por eso no se reportan media/mediana numérica.
    
*   **Categóricas (moda):**
    
    *   rating → **NaN** (faltante) con **74.94%** de los registros: confirma **alta ausencia** en certificaciones, coherente con evitar esta variable.
        
    *   type → **Theatrical** con **56.28%**: predomina el **estreno en cines** frente a otros tipos.
        
    *   country → **USA** con **24.08%**: EE. UU. concentra la mayor cantidad de **eventos de estreno** (recuerda que releases puede tener **múltiples filas por película**).
        
    *   date → **2006-01-01** con **0.23%**: la moda diaria es poco informativa por la **gran dispersión de fechas**.
        
*   **Fecha (mediana):** **2012-10-23**. Es un indicador más robusto del **centro temporal** del conjunto (muestra fuerte peso en los 2000s–2010s).
    

**Genres**

*   **Numéricas:** no aplican.
    
*   **Categórica (moda):** genre → **Drama** con **22.18%** de las asignaciones. Importante: genres es **multi-etiqueta** (una película puede aportar a varios géneros), así que la moda refleja **etiquetas más frecuentes**, no “género principal”.
    

**Countries**

*   **Numéricas:** no aplican.
    
*   **Categórica (moda):** country → **USA** con **25.16%** de las asociaciones. Este dataset también es **multi-país** (una película puede tener varios países asociados).
    

**Lecturas clave para el proyecto**

*   La **altísima falta de rating** en releases justifica excluir esa columna.
    
*   EE. UU. aparece como **modo** tanto en releases.country como en countries.country, lo que confirma **alto volumen** para trabajar la hipótesis centrada en Estados Unidos.
    
*   La **mediana de date en 2012** sugiere que las décadas **2000–2009** y **2010–2019** tienen **mucha masa de datos**, alineado con el análisis por décadas.

## Medidas de dispersión por dataset

In [29]:
# Medidas de dispersión por dataset: varianza, desviación estándar, rango, coeficiente de variación, IQR
import pandas as pd
import numpy as np
from IPython.display import display

def _numeric(df):
    """Selecciona columnas numéricas relevantes (excluye 'id')."""
    return df.select_dtypes(include=[np.number]).drop(columns=["id"], errors="ignore")

def _metrics(series: pd.Series) -> pd.Series:
    """Calcula var, std, rango, coef. variación, IQR para una serie numérica."""
    s = series.dropna()
    if s.empty:
        return pd.Series({"var": np.nan, "std": np.nan, "range": np.nan, "cv": np.nan, "iqr": np.nan})
    var = s.var()  # ddof=1 (muestra)
    std = s.std()  # ddof=1
    rng = s.max() - s.min()
    mean = s.mean()
    with np.errstate(divide="ignore", invalid="ignore"):
        cv = std / mean if mean not in (0, np.nan) else np.nan
    q75, q25 = s.quantile(0.75), s.quantile(0.25)
    iqr = q75 - q25
    return pd.Series({"var": var, "std": std, "range": rng, "cv": cv, "iqr": iqr})

def dispersion_table(df: pd.DataFrame) -> pd.DataFrame:
    """Tabla de dispersión para todas las columnas numéricas (excluyendo id)."""
    num = _numeric(df)
    if num.shape[1] == 0:
        return pd.DataFrame({"note": ["(sin columnas numéricas relevantes)"]})
    rows = []
    for col in num.columns:
        m = _metrics(num[col])
        m.name = col
        rows.append(m)
    return pd.DataFrame(rows).round(6)

def _date_to_days(df: pd.DataFrame, date_col: str):
    """Convierte una columna de fecha a días (float) para calcular dispersión temporal."""
    if date_col not in df.columns:
        return None
    s = pd.to_datetime(df[date_col], errors="coerce", utc=True).dt.tz_localize(None).dropna()
    if s.empty:
        return None
    # ns -> s -> días (uso astype para evitar FutureWarning)
    days = (s.astype("int64") / 1e9) / 86400.0
    return pd.Series(days, index=s.index)

# DataFrames ya cargados en tu sesión:
releases  = dfs["releases"].copy()
genres    = dfs["genres"].copy()
countries = dfs["countries"].copy()

print("="*40)
print("=== DISPERSIÓN — releases (numéricas) ===")
print("="*40)
display(dispersion_table(releases))

# Extra: dispersión temporal de 'date' en días (útil porque suele no haber numéricas aparte de 'id')
print("="*40)
print("\n=== DISPERSIÓN — releases.fecha (en días) ===")
print("="*40)
rel_days = _date_to_days(releases, "date")
if rel_days is not None:
    display(pd.DataFrame([_metrics(rel_days)], index=["date_days"]).round(6))
else:
    print("(columna 'date' no parseable o ausente)")

print("="*40)
print("\n=== DISPERSIÓN — genres (numéricas) ===")
print("="*40)
display(dispersion_table(genres))

print("="*40)
print("\n=== DISPERSIÓN — countries (numéricas) ===")
print("="*40)
display(dispersion_table(countries))


=== DISPERSIÓN — releases (numéricas) ===


,note
0,(sin columnas numéricas relevantes)



=== DISPERSIÓN — releases.fecha (en días) ===


,var,std,range,cv,iqr
date_days,8.405585e+07,9168.197483,82180.0,0.76722,9190.0



=== DISPERSIÓN — genres (numéricas) ===


,note
0,(sin columnas numéricas relevantes)



=== DISPERSIÓN — countries (numéricas) ===


,note
0,(sin columnas numéricas relevantes)


**Releases (fecha en días)**

*   **Rango:** 82 180 días ≈ **225.2 años** → hay estrenos muy antiguos y muy recientes; la ventana temporal es **extremadamente amplia**.
    
*   **Desviación estándar:** 9 168.20 días ≈ **25.1 años** → alta variabilidad temporal.
    
*   **IQR (P75–P25):** 9 190 días ≈ **25.2 años** → el 50% central de fechas también está muy disperso.
    
*   **Varianza:** 8.41×10⁷ días² → consistente con la gran dispersión.
    
*   **Coeficiente de variación (CV ≈ 0.77):** calculado sobre “días desde epoch”; sirve como indicador relativo, pero **es menos intuitivo** para fechas. Úsalo solo como referencia.
    

**Genres / Countries**

*   No hay columnas numéricas (excluimos id), por eso no se reportan varianza, desviación, rango, CV e IQR para estos datasets.
    

**Conclusiones**

*   Los estrenos cubren **muchas décadas**, lo que explica la dispersión tan alta. Si el análisis se centra en **2000–2019**, esto **se recortará en Fase 3 (Preparación)**; por ahora, en Fase 2, dejamos documentado el hallazgo.

## Medidas de posición por dataset

In [30]:
# Medidas de posición por dataset:
# - Numéricas: conteo (non-null), suma, mínimo, p10, p25, mediana, p75, p90, máximo
# - Fechas (releases.date): conteo válido, mínimo, p10, p25, mediana, p75, p90, máximo
import pandas as pd
import numpy as np
from IPython.display import display

def _numeric(df):
    """Selecciona columnas numéricas relevantes (excluye 'id')."""
    return df.select_dtypes(include=[np.number]).drop(columns=["id"], errors="ignore")

def numeric_position_table(df: pd.DataFrame) -> pd.DataFrame:
    num = _numeric(df)
    if num.shape[1] == 0:
        return pd.DataFrame({"note": ["(sin columnas numéricas relevantes)"]})
    desc = num.describe(percentiles=[.10, .25, .50, .75, .90]).T
    # Renombrar percentiles para claridad y agregar suma total
    desc = desc.rename(columns={"count":"count", "mean":"mean", "std":"std", "min":"min",
                               "10%":"p10", "25%":"p25", "50%":"median", "75%":"p75", "90%":"p90", "max":"max"})
    desc["sum"] = num.sum()
    # Ordenar columnas típicas de posición
    cols = ["count", "sum", "min", "p10", "p25", "median", "p75", "p90", "max"]
    return desc[cols].round(6)

def date_position_table(df: pd.DataFrame, date_col: str) -> pd.DataFrame:
    if date_col not in df.columns:
        return pd.DataFrame({"note": [f"(columna '{date_col}' no existe)"]})
    s = pd.to_datetime(df[date_col], errors="coerce", utc=True).dt.tz_localize(None).dropna()
    if s.empty:
        return pd.DataFrame({"note": [f"(no parseable '{date_col}')"]})
    qs = s.quantile([.10, .25, .50, .75, .90])
    out = pd.DataFrame({
        "count_valid": [s.shape[0]],
        "min": [s.min()],
        "p10": [qs.loc[0.10]],
        "p25": [qs.loc[0.25]],
        "median": [qs.loc[0.50]],
        "p75": [qs.loc[0.75]],
        "p90": [qs.loc[0.90]],
        "max": [s.max()]
    }, index=[date_col])
    return out

# DataFrames ya cargados en tu sesión
releases  = dfs["releases"].copy()
genres    = dfs["genres"].copy()
countries = dfs["countries"].copy()

print("========================================")
print("=== POSICIÓN — releases (numéricas) ===")
print("========================================")
display(numeric_position_table(releases))

print("\n=== POSICIÓN — releases.fecha (date) ===")
display(date_position_table(releases, "date"))

print("\n=======================================")
print("=== POSICIÓN — genres (numéricas) ===")
print("=======================================")
display(numeric_position_table(genres))

print("\n=========================================")
print("=== POSICIÓN — countries (numéricas) ===")
print("=========================================")
display(numeric_position_table(countries))


=== POSICIÓN — releases (numéricas) ===


,note
0,(sin columnas numéricas relevantes)



=== POSICIÓN — releases.fecha (date) ===


,count_valid,min,p10,p25,median,p75,p90,max
date,1332782,1874-12-09,1965-10-07,1994-11-05,2012-10-23,2020-01-03,2022-12-23,2099-12-09



=== POSICIÓN — genres (numéricas) ===


,note
0,(sin columnas numéricas relevantes)



=== POSICIÓN — countries (numéricas) ===


,note
0,(sin columnas numéricas relevantes)


**Releases (columna date)**

*   **count\_valid = 1,332,782**: todas las fechas se **pudieron parsear** (coincide con el total de filas de releases).
    
*   **mín = 1874-12-09**: fecha **muy anterior** al cine moderno → probable **outlier**/dato histórico mal registrado.
    
*   **p10 = 1965-10-07** y **p25 = 1994-11-05**: hay una cola de títulos **antiguos**.
    
*   **mediana = 2012-10-23**: el “centro” temporal cae en los **2010s**, consistente con mayor densidad reciente.
    
*   **p75 = 2020-01-03** y **p90 = 2022-12-23**: hay **bastante volumen** posterior a 2019.
    
*   **máx = 2099-12-09**: fecha **futura** → casi seguro **valor anómalo**/placeholder.
    

**Genres / Countries**

*   No hay columnas numéricas (aparte de id, que excluimos), así que **mín/máx/suma/conteo numéricos no aplican** en estos dos datasets. Para ellos, las medidas útiles son las **categóricas** (p. ej., moda) que ya reportaste.
    

**Conclusiones**

*   El rango temporal de releases es **extremadamente amplio** (1874–2099), con mediana en 2012: describe bien la **posición** del conjunto.
    
*   Se identifican **outliers temporales** (muy antiguos y futuros). **Solo se documenta** en Fase 2; el recorte a 2000–2019 o el tratamiento de outliers corresponderá a **Fase 3 (Preparación de datos)**.

# Verificar calidad de los datos

## Hallazgos por dataset

### 1) releases (id, country, date, type, rating)

*   **Fechas (validez):** date es **parseable** (count\_valid = 1,332,782).
    
    *   **Mín:** 1874-12-09 (muy antigua)
        
    *   **Mediana:** 2012-10-23
        
    *   **Máx:** 2099-12-09 (**futura**, probable placeholder)➜ **Riesgo:** rango temporal **excesivo** con outliers antiguos y futuros.
        
*   **Completitud:** rating tiene ~**74.94%** nulos (moda = NaN).➜ **Decisión:** **excluir** rating del análisis (no es fiable).
    
*   **Consistencia categórica:**
    
    *   type moda = **Theatrical** (valores esperados, p. ej. Theatrical/Digital).
        
    *   country incluye variantes de EE. UU. (p. ej., _USA/US/United States_).➜ **Necesita normalización** de países (documentado, sin ejecutar aún).
        
*   **Unicidad esperada (claves lógicas a auditar):**
    
    *   Por evento de estreno: **(id, country, type, date)**.➜ Verificar duplicados exactos sobre esa clave (documentado).
        

### 2) genres (id, genre)

*   **Completitud:** sin nulos relevantes reportados.
    
*   **Categórica:** moda = **Drama** (~22.18%).
    
*   **Naturaleza:** relación **multi-etiqueta** (varios géneros por película).
    
*   **Unicidad esperada:** **(id, genre)** sin duplicados exactos.➜ Auditar duplicados exactos y, si existen, documentarlos.
    

### 3) countries (id, country)

*   **Completitud:** sin nulos relevantes reportados.
    
*   **Categórica:** moda = **USA** (~25.16%).
    
*   **Naturaleza:** relación **multi-país** (varios países por película).
    
*   **Consistencia:** variantes de EE. UU. (USA/US/United States).
    
*   **Unicidad esperada:** **(id, country)** sin duplicados exactos.➜ Auditar duplicados exactos.

## Conclusión de calidad
------------------------------

*   Los datasets **son aptos** para continuar con el análisis, con las siguientes **condiciones** ya identificadas:
    
    1.  **Excluir rating** (alta ausencia).
        
    2.  **Filtrar fechas a 2000–2019** y usar **primera fecha por película** para evitar reestrenos múltiples.
        
    3.  **Normalizar** el país “Estados Unidos” y otras variantes.
        
    4.  **Verificar/deduplicar** claves lógicas: releases (id, country, type, date), genres (id, genre), countries (id, country).
        
*   Estas acciones se **documentan ahora** y se **ejecutarán en Fase 3 (Preparación de datos)**.